In [ ]:
import aggregation as aggregation
import nltk as nltk
import pandas as pd
import numpy as np
#plotting
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
import matplotlib.dates as mdates
import plotly.express as px
from plotnine import *
#from ggplot import *



#Sentimentanalysis - Dictionary Approach
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


import nest_asyncio
import re
from matplotlib import pyplot as plt

import time
from datetime import datetime, date, time, timedelta
import dateutil.relativedelta

In [ ]:
df1 = pd.read_excel('../../DataSources/replies_with_media_war.xlsx')

In [ ]:
df1 = df1[df1.lang == 'en']

In [ ]:
def preprocess_tweets(text):

    fo = open("Stopwords-en.txt","+r")
    stop_words = list(fo.read().split(','))
    translation={39:None}
    processed_tweet = text
    processed_tweet=' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())
    processed_tweet = " ".join(word for word in processed_tweet.split() if word not in str(stop_words).translate(translation))
    return(processed_tweet)

df1['Processed Tweet'] = df1['text'].apply(lambda x: preprocess_tweets(x.lower()))

In [ ]:
#Clean Rows where Processed Tweet is Empty (e.g. because Reply is only a Link)
def EmptyRows(dataset):
    nan_value = float("NaN")
    dataset.replace("",nan_value, inplace=True)
    dataset.dropna(subset = ['Processed Tweet'], inplace= True)

In [ ]:
def SplitDate(dataset):
    dataset['year']=pd.to_datetime(dataset['date']).dt.strftime('%Y')
    dataset['month']=pd.to_datetime(dataset['date']).dt.strftime('%m')
    dataset['day']=pd.to_datetime(dataset['date']).dt.strftime('%A')
    dataset['year-month']=pd.to_datetime(dataset['date']).dt.strftime('%Y-%m')

In [ ]:
EmptyRows(df1)

In [ ]:
df1['polarity'] = df1['Processed Tweet'].apply(lambda x: analyzer.polarity_scores(x))

In [ ]:
def sentimentPredict(sentiment):
    if sentiment['compound'] >= 0.05:
        return "Positive"
    elif sentiment['compound'] <= -0.05:
        return "Negative"
    else:
        return "Neutral"

df1['sentiment'] =df1['polarity'].apply(lambda x: sentimentPredict(x))



In [ ]:
#for creating columns for eacht dic item in polarity
df1 = pd.concat([df1.drop(['polarity'], axis=1), df1['polarity'].apply(pd.Series)], axis=1)

In [ ]:
plt.figure(figsize=(6,5))
plt.title('Classification of All Replys during ukraine war into sentiment categories',fontsize=15)
plt.ylabel('Percentage [%]',fontsize=18)
ax = (df1.sentiment.value_counts()/len(df1)*100).plot(kind="bar", rot=0,color=['#04407F','#0656AC','#0A73E1'])
ax.set_yticks(np.arange(0, 110, 10))
plt.grid(color='#95a5a6', linestyle='-.', linewidth=1, axis='y', alpha=0.7)
ax2 = ax.twinx()
ax2.set_yticks(np.arange(0, 110, 10)*len(df1)/100)
for p in ax.patches:
    ax.annotate('{:.2f}%'.format(p.get_height()), (p.get_x()+0.15, p.get_height()+1))

In [ ]:
#df1.to_excel('check_words.xlsx')

In [ ]:
cross_tab_prop2 = pd.crosstab(index=df1['before_after'], columns=df1['sentiment'], normalize='index')
cross_tab_prop2.plot(kind='bar', stacked=True, figsize=(18, 12))
plt.legend(loc="upper left", ncol=2)
plt.title('Relative distribution of Sentiments in User Replys during ukraine war', fontsize=15)
plt.xlabel("Year")
plt.ylabel("Proportion")
plt.xticks(rotation=45)

for n, x in enumerate([*cross_tab_prop2.index.values]):
    for (proportion, y_loc) in zip(cross_tab_prop2.loc[x],
                                   cross_tab_prop2.loc[x].cumsum()):
        plt.text(x=n - 0.17,
                 y=y_loc,
                 s=f'{np.round(proportion * 100, 1)}%',
                 color="black",
                 fontsize=20,
                 fontweight="bold")

plt.show()

In [ ]:
cross_tab_prop2

In [15]:
#cross_tab_prop2.to_csv("before_after_replys.csv")

In [ ]:
#Testing compound score for ukraine invasion related words like war and weapon
sid = SentimentIntensityAnalyzer()
a= 'war'
b = 'weapon'

In [ ]:
sid.polarity_scores(a)